Adicionar o path de Dados do Google Drive

In [1]:
import plotly.express as px
import pandas as pd
import gspread
import glob

Carregar os Dados da Planilha online Google Sheets

In [2]:
class GoogleSheets:
    def __init__(self):
        self.key = glob.glob("key/*.json")[0]
        self.CODE = '1AYpQ_uHm7aZ-VZwibuscpQxgJ-vXirm4PWrJDECYr1c'
        self.gc = gspread.service_account(filename=self.key)
        self.sh = self.gc.open_by_key(self.CODE)

    def visualizar_dados(self):
        ws = self.sh.worksheet('CONVIDADOS DO CASAMENTO')
        dados = ws.get_all_records()
        planilha = pd.DataFrame(dados)
        return planilha
    
    def transformar_minuscula(self,coluna):
        coluna_capitalizada = coluna.str.title()
        return coluna_capitalizada
    
    def verificar_erros_gramatica(self,coluna):
        erros_gramatica = coluna[coluna.str.contains(r'\b(?:[A-Z]{2,}|[a-z]{2,})\b')]
        return erros_gramatica
    
# Create an instance of the GoogleSheets class
googlesheet = GoogleSheets()

df=googlesheet.visualizar_dados()
df["Nome"] = googlesheet.transformar_minuscula(df["Nome"])
df["Convite Enviado ?"] = df["Convite Enviado ?"].fillna("Não informado")
df

,Nome,Parentesco,Padrinho,Localidade:,Criança,Convite Enviado ?,Codigo
0,Ana Silva,Família,,,Não,Sim,64ddc
1,Bruno Santos,Amigo,,,Não,Sim,6606c
2,Carlos Oliveira,Trabalho,,,Não,Sim,673c1
3,Daniela Souza,Família,Sim,,Não,Sim,6894f
4,Eduardo Pereira,Amigo,,,Não,Sim,69eef
...,...,...,...,...,...,...,...
152,Carlos Almeida,Família,,,Não,Sim,8841f
153,Daniela Nascimento,Trabalho,,,Não,Sim,f3cf5
154,Eduardo Lima,Amigo,,,Não,Sim,f899
155,Fernanda Gomes,Família,,,Não,Sim,2359y


Criar vizualização para ver os convites enviados

In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.Set2

def grafico_top_n(df, coluna, titulo, top_n=5):
    
    # Contagem
    dados = df[coluna].value_counts().reset_index()
    dados.columns = ['Categoria', 'Quantidade']
    
    # Separar Top N
    top = dados.head(top_n)
    outros = dados.iloc[top_n:]
    
    # Agrupar como "Outros"
    if len(outros) > 0:
        outros_total = outros['Quantidade'].sum()
        
        linha_outros = pd.DataFrame({
            'Categoria': ['Outros'],
            'Quantidade': [outros_total]
        })
        
        top = pd.concat([top, linha_outros], ignore_index=True)
    
    # Criar gráfico
    fig = go.Figure(
        go.Pie(
            labels=top['Categoria'],
            values=top['Quantidade'],
            hole=0.55,
            marker=dict(colors=colors[:len(top)]),
            textinfo='percent+label',
            hovertemplate="<b>%{label}</b><br>Qtd: %{value}<br>Percentual: %{percent}<extra></extra>"
        )
    )
    
    fig.update_layout(
        title={'text': titulo, 'x': 0.5},
        template='plotly_white',
        height=500,
        width=700
    )
    
    fig.show()


# Gerar gráficos
grafico_top_n(df, 'Convite Enviado ?', 'Convites Enviados', top_n=3)
grafico_top_n(df, 'Parentesco', 'Parentesco (Top 5)', top_n=5)
grafico_top_n(df, 'Localidade:', 'Localidade (Top 5)', top_n=5)


Gerar os Convites

In [7]:
from PIL import Image, ImageDraw, ImageFont
import shutil
import os
import pandas as pd
import uuid


# Classe do Sistema
class Sistema:
    @staticmethod
    def apagar_conteudo_diretorio(diretorio):
        if os.path.exists(diretorio):
            for item in os.listdir(diretorio):
                item_path = os.path.join(diretorio, item)
                try:
                    if os.path.isfile(item_path) or os.path.islink(item_path):
                        os.unlink(item_path)  # Remove arquivos ou links simbólicos
                    elif os.path.isdir(item_path):
                        shutil.rmtree(item_path)  # Remove diretórios recursivamente
                except Exception as e:
                    print(f'Erro ao apagar {item_path}: {e}')
            print(f'Todo o conteúdo do diretório {diretorio} foi apagado com sucesso.')
        else:
            print(f'O diretório {diretorio} não existe.')

# Exemplo de uso
diretorio_para_limpar = 'convites_gerados'
Sistema.apagar_conteudo_diretorio(diretorio_para_limpar)

# Definir fontes
font1 = ImageFont.truetype("font/font.otf", size=45)
font2 = ImageFont.truetype("font/font.otf", size=30)
path_imagem_base = "template/template.png"
path_imagem_output = "convites_gerados"

def gerar_convite(convidado, codigo,path_imagem_base,path_imagem_output,font1,font2):

    imagem = Image.open(path_imagem_base).convert("RGBA")

    lapis = ImageDraw.Draw(imagem)

    dados_imagem = imagem.size
    dados_texto = len(convidado)

    print(f"Pessoa: {convidado}, Tamanho do texto: {dados_texto}")

    # Calcular a posição x para centralizar o texto
    # Use getbbox instead of getsize
    posicao_x = (dados_imagem[0] - font1.getbbox(convidado)[2]) // 2

    # Nome do Convidado
    lapis.text((posicao_x, 1170), text=convidado, fill="#000", font=font1) # Update to use posicao_x

    # Código do Convite
    lapis.text((165, 1265), text=f"código: {codigo}", fill="#000", font=font2)

    # Salvar a imagem com o nome do convidado.
    imagem.save(f"{path_imagem_output}/CONVITE - {convidado}.png")

for convidado in df.values:
    gerar_convite(convidado[0], convidado[6],path_imagem_base,path_imagem_output,font1,font2)


Erro ao apagar convites_gerados\CONVITE - Gabriel Rodrigues.png: [WinError 32] O arquivo já está sendo usado por outro processo: 'convites_gerados\\CONVITE - Gabriel Rodrigues.png'
Erro ao apagar convites_gerados\CONVITE - Lucas Gomes.png: [WinError 32] O arquivo já está sendo usado por outro processo: 'convites_gerados\\CONVITE - Lucas Gomes.png'
Todo o conteúdo do diretório convites_gerados foi apagado com sucesso.
Pessoa: Ana Silva, Tamanho do texto: 9
Pessoa: Bruno Santos, Tamanho do texto: 12
Pessoa: Carlos Oliveira, Tamanho do texto: 15
Pessoa: Daniela Souza, Tamanho do texto: 13
Pessoa: Eduardo Pereira, Tamanho do texto: 15


KeyboardInterrupt: 